# Relevamiento de carga de datos sospechosa en las elecciones de Provincia de Buenos Aires

Este análisis es mi aporte para hacer el trabajo de **chequeo manual de telegramas de las PASO 2017** en la provincia de Buenos Aires de una más eficientemente, detectando aquellas resultados estádisticamente raros (no quiere decir que todas las mesas listadas estén incorrectas). 

Para esto realizé un programa que obtiene los datos digitalizados de cada una de las mesas y los estructuré en una base de datos para hacer agregaciones y consultas. 

Por ejemplo, basado en la idea de que los resultados en una mesa son similares en un circuito, se pueden detectar aquellas mesas que tienen resultados muy distintos. 

Las tablas resultantes muestran algunos datos e incluyen el link para ver el telegrama en el sitio oficial para que revisemos entre tod@s. 

Todo el [código fuente está disponible en github](http://github.com/mgaitan/mesas_ba/)

**Gracias por difundir**

- Martín Gaitán [`@tin_nqn_`](https://twitter.com/tin_nqn_)



<div class="alert alert-info" role="alert">Si querés podés <a href="#resultados">ir a los resultados</a>




</div>



### Generación de base de datos

Utilizo el ORM de Django para modelar la base de datos y luego importo los CSVs del [crawler del escrutinio provisorio](https://github.com/mgaitan/mesas_ba/tree/master/mesas_ba), mesa por mesa. 


In [1]:
!pip install django-orm-magic

  Running setup.py bdist_wheel for django-orm-magic ... done
  Stored in directory: /home/tin/.cache/pip/wheels/6e/45/52/31f3c132bc0fb005bebab9482133db4a9a5cadb98e7684551b
Successfully built django-orm-magic


In [1]:
%load_ext django_orm_magic

In [2]:
%%django_orm
from django.db import models
from django.db.models import Sum

class Seccion(models.Model):
    numero = models.CharField(max_length=10)
    nombre = models.CharField(max_length=100)
    
    def __str__(self):
        return f'{self.numero} - {self.nombre}'

class Circuito(models.Model):
    numero = models.CharField(max_length=10)
    seccion = models.ForeignKey('Seccion')

    def __str__(self):
        return u"Circuito %s (%s)" % (self.numero, self.seccion)
    
class Mesa(models.Model):
    numero = models.CharField(max_length=10)
    circuito = models.ForeignKey('Circuito')
    fake_id = models.CharField(max_length=100, unique=True)
    url = models.CharField(max_length=200)
    
    class Meta:
        unique_together = ('circuito', 'numero')
    
    @property
    def computados_sen(self):
        return self.resultados.filter(categoria='sen').aggregate(Sum('votos'))['votos__sum']
    
    @property
    def computados_dip(self):
        return self.resultados.filter(categoria='dip').aggregate(Sum('votos'))['votos__sum']
    
    @property
    def computados(self):
        # aprox
        return (self.computados_sen + self.computados_dip) // 2


    def __str__(self):
        return u"Mesa %s (%s)" % (self.numero, self.circuito)
    
    @property
    def absolute_url(self):
        return f'http://www.resultados.gob.ar/99/resu/content/telegramas/{self.url}'
    

class Partido(models.Model):
    nombre = models.CharField(max_length=100, unique=True)
        
    def __str__(self):
        return self.nombre
    
    
class Opcion(models.Model):
    nombre = models.CharField(max_length=100, unique=True)
    partido = models.ForeignKey('Partido', null=True)    # blanco / nulo / etc. 
    
    def __str__(self):
        return self.nombre
    
class VotoMesa(models.Model):
    mesa = models.ForeignKey('Mesa', related_name='resultados')
    categoria = models.CharField(max_length=20, choices=(('dip', 'Diputados'), ('sen', 'Senadores')))
    opcion = models.ForeignKey('Opcion')
    votos = models.IntegerField()
    
    def __str__(self):
        return u"%s: %s -> %d" % (self.get_categoria_display(), self.opcion, self.votos)
    
    
    class Meta:
        unique_together = ('mesa', 'opcion', 'categoria')

## Importación de datos a los modelos

La [base de datos está disponible](http://lab.nqnwebs.com/descargas/db.sqlite) (Md5sum: `0e5d3f0c92ba8d18fc19b7a3ed742266`) Teniendo esta base **no hace falta que repitas el scraping e importes los datos** Podés salterte esto.

In [5]:
from csv import DictReader

In [6]:
Seccion.objects.all()

<QuerySet []>

Importo secciones, circuitos y mesas de los distintos archivos

In [7]:
for seccion in DictReader(open('secciones.csv')):
    Seccion.objects.create(**seccion)
    

In [8]:
secciones = {s.numero: s for s in Seccion.objects.all()}
circuitos = []
for circuito in DictReader(open('circuitos.csv')):
    circuito['seccion'] = secciones[circuito['seccion']] 
    circuitos.append(Circuito(**circuito))
Circuito.objects.bulk_create(circuitos);

In [9]:
circuitos = {c.numero: c for c in Circuito.objects.all()}
mesas = []
for mesa in DictReader(open('mesas.csv')):
    mesa['circuito'] = circuitos[mesa['circuito']]
    mesas.append(Mesa(**mesa))
Mesa.objects.bulk_create(mesas);

El archivo `resultados.csv` pesa casi 100mb y tiene una fila por cada mesa-opcion, para senadores y diputados. 
Para cargarlo más rapido (sino puede tardar muchisimo) hice unas pequeñas optimizaciones. 

- Importo partidos y listas que existan primero y armo un diccionario para no tener que hacer consultas luego
- Hago "chunks" e inserto varias filas de resultados a la vez. 

Con esto tarda unos 5' en cargar. 

In [10]:
from itertools import zip_longest

def grouper(n, iterable, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

In [15]:
import pandas as pd
partidos_listas = pd.read_csv('resultados.csv', usecols=['partido','lista'])
partidos_listas.drop_duplicates(inplace=True)


,lista,partido
0,NaN,nulos
2,NaN,blancos
4,NaN,recurridos
6,NaN,impuganados
8,ROJO PUNZO,FEDERAL
10,CELESTE Y BLANCA,FEDERAL
12,FEDERALISMO PURO,FEDERAL
14,ALTERNATIVA,FEDERAL
31,NARANJA,HUMANISTA
33,INDEPENDENCIA,DEL CAMPO POPULAR


In [31]:
partidos_listas.fillna('', inplace=True)
partidos_listas

,lista,partido
0,,nulos
2,,blancos
4,,recurridos
6,,impuganados
8,ROJO PUNZO,FEDERAL
10,CELESTE Y BLANCA,FEDERAL
12,FEDERALISMO PURO,FEDERAL
14,ALTERNATIVA,FEDERAL
31,NARANJA,HUMANISTA
33,INDEPENDENCIA,DEL CAMPO POPULAR


In [32]:
Partido.objects.all().delete()

(0, {})

In [33]:
for _, row in partidos_listas.iterrows():
    if not row.lista:
        Opcion.objects.create(nombre=row.partido)
    else:
        partido, _ = Partido.objects.get_or_create(nombre=row.partido)
        Opcion.objects.create(nombre=row.lista, partido=partido)

In [34]:
opciones = {o.nombre: o for o in Opcion.objects.all()}
opciones

{'ALTERNATIVA': <Opcion: ALTERNATIVA>,
 'CAMBIANDO JUNTOS': <Opcion: CAMBIANDO JUNTOS>,
 'CELESTE Y BLANCA': <Opcion: CELESTE Y BLANCA>,
 'CELESTE Y BLANCA U': <Opcion: CELESTE Y BLANCA U>,
 'COMPROMISO CIUDADANO': <Opcion: COMPROMISO CIUDADANO>,
 'CONDUCCION': <Opcion: CONDUCCION>,
 'CONVICCIONES': <Opcion: CONVICCIONES>,
 'CUMPLIR': <Opcion: CUMPLIR>,
 'ESPERANZA': <Opcion: ESPERANZA>,
 'FEDERALISMO PURO': <Opcion: FEDERALISMO PURO>,
 'INDEPENDENCIA': <Opcion: INDEPENDENCIA>,
 'LIBERTAD': <Opcion: LIBERTAD>,
 'LISTA 1': <Opcion: LISTA 1>,
 'LISTA A': <Opcion: LISTA A>,
 'MEMORIA, VERDAD Y JUSTICIA': <Opcion: MEMORIA, VERDAD Y JUSTICIA>,
 'NARANJA': <Opcion: NARANJA>,
 'ORDEN  Y SEGURIDAD': <Opcion: ORDEN  Y SEGURIDAD>,
 'PIONEROS': <Opcion: PIONEROS>,
 'ROJO PUNZO': <Opcion: ROJO PUNZO>,
 'SEGURIDAD': <Opcion: SEGURIDAD>,
 'SOBERANIA': <Opcion: SOBERANIA>,
 'SOLIDARIDAD': <Opcion: SOLIDARIDAD>,
 'TOBA': <Opcion: TOBA>,
 'UN PAIS UNIDO HOY Y SIEMPRE': <Opcion: UN PAIS UNIDO HOY Y SIEM

In [36]:
mesas = {m.fake_id: m for m in Mesa.objects.all()}

In [ ]:
for chunk in grouper(250, DictReader(open('resultados.csv'))):
    resultados = []
    for resultado in chunk:
        if not resultado:
            continue
        mesa = mesas[resultado['mesa']]
        lista = resultado['lista'] or resultado['partido']
        opcion = opciones[lista]
        try:
            votos = int(resultado['votos'])
        except ValueError:
            print('**votos no validos', resultado)
            continue
        
        resultados.append(VotoMesa(mesa=mesa, 
                               opcion=opcion, 
                               votos=votos, 
                               categoria=resultado['categoria']))
    VotoMesa.objects.bulk_create(resultados)

<div id="resultados"></div>

## RESULTADOS 



Teniendo la base de datos puedo hacer cualquier tipo de consulta, por mesa, por circuito, totales, etc. 

Primero calculemos los porcentajes totales para diputados y senadores

In [5]:
UC = Opcion.objects.filter(partido__nombre='UNIDAD CIUDADANA')[0]
UC, UC.partido

(<Opcion: CELESTE Y BLANCA U>, <Partido: UNIDAD CIUDADANA>)

In [6]:
CAMBIEMOS = Opcion.objects.get(id=32)
CAMBIEMOS, CAMBIEMOS.partido

(<Opcion: CAMBIANDO JUNTOS>, <Partido: CAMBIEMOS BUENOS AIRES>)

In [7]:
total_positivos = VotoMesa.objects.filter(opcion__partido__isnull=False, categoria='sen').aggregate(v=Sum('votos'))['v']
total_positivos

8601410

In [8]:
from django.db.models import Avg, Sum

In [9]:
VotoMesa.objects.filter(opcion=UC, categoria='sen').aggregate(v=Sum('votos'))['v']/total_positivos

0.3533368366349238

In [10]:
VotoMesa.objects.filter(opcion=CAMBIEMOS, categoria='sen').aggregate(v=Sum('votos'))['v']/total_positivos

0.35414077459393284

Según los cálculos, **Unidad Ciudadana tiene 35.33 %** y **Cambiemos el 35.41%** para senadores. Un 0.08% de diferencia

In [11]:
total_positivos_dip = VotoMesa.objects.filter(opcion__partido__isnull=False, categoria='dip').aggregate(v=Sum('votos'))['v']
total_positivos_dip

8438373

In [12]:
VotoMesa.objects.filter(opcion=UC, categoria='dip').aggregate(v=Sum('votos'))['v']/total_positivos_dip

0.3407902210532765

In [13]:
VotoMesa.objects.filter(opcion=CAMBIEMOS, categoria='dip').aggregate(v=Sum('votos'))['v']/total_positivos_dip

0.364079070692893

Para diputados **Unidad Ciudadana tiene 34.07 %** y **Cambiemos el 36.40%**

### Mesas donde el UC sacó menos del 50% de los votos promedio a Senador en el circuito

In [14]:
from IPython.display import display_html, HTML

In [15]:
mesas = []
for circuito in Circuito.objects.all():
    promedio_uc_circuito = VotoMesa.objects.filter(mesa__circuito=circuito, opcion=UC, categoria='sen').aggregate(p=Avg('votos'))['p']
    if not promedio_uc_circuito:
        continue
    
    for mesa in Mesa.objects.filter(circuito=circuito, resultados__votos__gt=0, 
                                    resultados__votos__lte=promedio_uc_circuito/2, 
                                    resultados__opcion=UC, resultados__categoria='sen'):
        vot_uc = mesa.resultados.get(opcion=UC, categoria='sen').votos
        
        mesas.append((mesa, vot_uc, promedio_uc_circuito))


In [16]:
len(mesas)

336

In [17]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Votos UC</th><th>Promedio UC/Circ</th></tr>'
for l in mesas:
    mesa, vot_uc, promedio_uc_circuito = l
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados_sen} </td><td>{vot_uc}</td><td> {promedio_uc_circuito:.2f}</td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa,Circuito,Computados,Votos UC,Promedio UC/Circ
Mesa 00107,Circuito 0789 (011 - Berazategui),85,39,98.13
Mesa 00267,Circuito 0789C (011 - Berazategui),190,40,84.37
Mesa 00256,Circuito 0789C (011 - Berazategui),197,4,84.37
Mesa 00336,Circuito 0789D (011 - Berazategui),133,1,121.37
Mesa 00573,Circuito 0792 (011 - Berazategui),58,15,101.40
Mesa 00526,Circuito 0792 (011 - Berazategui),317,18,101.40
Mesa 00727,Circuito 0792D (011 - Berazategui),156,62,125.36
Mesa 00726,Circuito 0792D (011 - Berazategui),145,58,125.36
Mesa 00067,Circuito 0117 (010 - Arrecifes),104,21,71.01
Mesa 00093,Circuito 0073 (007 - Bahía Blanca),146,20,40.85


### Mesas no computadas

In [28]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th></tr>'
for mesa in Mesa.objects.filter(resultados__votos=0).distinct():
    if mesa.resultados.all().aggregate(p=Avg('votos'))['p'] != 0:
        # no computadas
        continue        
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa,Circuito
Mesa 00025,Circuito 0668 (134 - Hurlingham)
Mesa 00953,Circuito 0022 (003 - Almirante Brown)
Mesa 01037,Circuito 0022 (003 - Almirante Brown)
Mesa 09037,Circuito 0022 (003 - Almirante Brown)
Mesa 09035,Circuito 0022 (003 - Almirante Brown)
Mesa 09036,Circuito 0022 (003 - Almirante Brown)
Mesa 09012,Circuito 0667 (133 - Ituzaingo)
Mesa 09010,Circuito 0667 (133 - Ituzaingo)
Mesa 09008,Circuito 0667 (133 - Ituzaingo)
Mesa 09006,Circuito 0667 (133 - Ituzaingo)


### Mesas para los que UC sacó 0 votos para Senador en mesas que se computaron

In [29]:
mesas = []

for vm in VotoMesa.objects.filter(opcion=UC, categoria='sen', votos=0):
    if vm.mesa.resultados.filter(categoria='sen').aggregate(p=Avg('votos'))['p'] == 0:
        # no computadas
        continue    
    promedio_circuito = VotoMesa.objects.exclude(id=vm.id).filter(mesa__circuito=vm.mesa.circuito, opcion=UC, categoria='sen').distinct().aggregate(p=Avg('votos'))['p']
    mesa = vm.mesa
    mesas.append((mesa, promedio_circuito))


In [30]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Promedio UC/Circ</th></tr>'
for mesa, promedio_circuito in mesas:
    promedio_circuito = f'{promedio_circuito:.2f}' if promedio_circuito else 'N/A' 
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados_sen} </td><td> {promedio_circuito} </td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa,Circuito,Computados,Promedio UC/Circ
Mesa 09011,Circuito 0667 (133 - Ituzaingo),2,70.95
Mesa 09009,Circuito 0667 (133 - Ituzaingo),1,70.95
Mesa 09007,Circuito 0667 (133 - Ituzaingo),6,70.95
Mesa 09005,Circuito 0667 (133 - Ituzaingo),2,70.95
Mesa 09004,Circuito 0667 (133 - Ituzaingo),3,70.95
Mesa 00396,Circuito 0667 (133 - Ituzaingo),281,70.95
Mesa 09013,Circuito 0666A (133 - Ituzaingo),1,98.35
Mesa 09033,Circuito 0022 (003 - Almirante Brown),6,96.00
Mesa 09017,Circuito 0668 (134 - Hurlingham),2,68.90
Mesa 09026,Circuito 0022 (003 - Almirante Brown),1,96.00


In [31]:
len(mesas), int(sum(l[1] for l in mesas if l[1]))

(597, 48646)

Sólo en estas mesas estarían faltando alrededor de 48mil votos a CFK, aunque muchas tienen errores generales de carga y tampoco le sumaron los votos a otros partidos. Considerando sólo las que evidentemente penalizaron más UC da 17500 votos no contabilizados:

In [32]:
int(sum(l[1] for l in mesas if l[1] and l[0].computados_sen > 99))

17582

### Mesas para los que UC sacó 0 votos para Diputados en mesas que se computaron

In [23]:
mesas = []

for vm in VotoMesa.objects.filter(opcion=UC, categoria='dip', votos=0):
    if vm.mesa.resultados.filter(categoria='dip').aggregate(p=Avg('votos'))['p'] == 0:
        # no computadas
        continue    
    promedio_circuito = VotoMesa.objects.exclude(id=vm.id).filter(mesa__circuito=vm.mesa.circuito, opcion=UC, categoria='dip').distinct().aggregate(p=Avg('votos'))['p']
    mesa = vm.mesa
    mesas.append((mesa, promedio_circuito))

    
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Promedio UC/Circ</th></tr>'
for mesa, promedio_circuito in mesas:
    promedio_circuito = f'{promedio_circuito:.2f}' if promedio_circuito else 'N/A' 
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados_dip} </td><td> {promedio_circuito} </td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa,Circuito,Computados,Promedio UC/Circ
Mesa 09011,Circuito 0667 (133 - Ituzaingo),2,68.68
Mesa 09009,Circuito 0667 (133 - Ituzaingo),1,68.68
Mesa 09007,Circuito 0667 (133 - Ituzaingo),6,68.68
Mesa 09005,Circuito 0667 (133 - Ituzaingo),2,68.68
Mesa 09004,Circuito 0667 (133 - Ituzaingo),3,68.68
Mesa 00396,Circuito 0667 (133 - Ituzaingo),281,68.68
Mesa 09013,Circuito 0666A (133 - Ituzaingo),1,94.73
Mesa 09033,Circuito 0022 (003 - Almirante Brown),6,90.75
Mesa 09017,Circuito 0668 (134 - Hurlingham),2,65.65
Mesa 09026,Circuito 0022 (003 - Almirante Brown),1,90.75


In [24]:
len(mesas), int(sum(l[1] for l in mesas if l[1]))

(632, 48692)

### Mesas para los que Cambiemos sacó 70% o más del promedio de su circuito

In [25]:
mesas = []
for circuito in Circuito.objects.all():
    promedio_circuito = VotoMesa.objects.filter(mesa__circuito=circuito, opcion=CAMBIEMOS, categoria='sen').aggregate(p=Avg('votos'))['p']
    if not promedio_circuito:
        continue
    
    for mesa in Mesa.objects.filter(circuito=circuito, resultados__votos__gt=0, 
                                    resultados__votos__gte=promedio_circuito*1.7, 
                                    resultados__opcion=CAMBIEMOS, resultados__categoria='sen'):
        vot = mesa.resultados.get(opcion=CAMBIEMOS, categoria='sen').votos
        
        mesas.append((mesa, vot, promedio_circuito))


In [26]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Votos CAMBIEMOS</th><th>Promedio CAMBIEMOS/Circ</th></tr>'
for l in mesas:
    mesa, vot, promedio_circuito = l
    html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados_sen} </td><td>{vot}</td><td> {promedio_circuito:.2f}</td></tr>"
html += '</table>'
    
display_html(HTML(html))

Mesa,Circuito,Computados,Votos CAMBIEMOS,Promedio CAMBIEMOS/Circ
Mesa 00093,Circuito 0789 (011 - Berazategui),283,89,48.60
Mesa 00079,Circuito 0789 (011 - Berazategui),275,86,48.60
Mesa 00061,Circuito 0789 (011 - Berazategui),288,88,48.60
Mesa 00081,Circuito 0110 (009 - Baradero),94,38,19.00
Mesa 00198,Circuito 0031 (004 - Avellaneda),290,111,55.00
Mesa 00200,Circuito 0031 (004 - Avellaneda),269,103,55.00
Mesa 00199,Circuito 0031 (004 - Avellaneda),273,98,55.00
Mesa 00024,Circuito 0011 (002 - Alberti),92,21,10.50
Mesa 00160,Circuito 0218 (031 - Chivilcoy),139,50,25.00
Mesa 00040,Circuito 0668 (134 - Hurlingham),333,193,94.43


### Mesas con más de 30% de corte de boleta para UC

In [27]:
html = '<table class="table"><tr><th>Mesa</th><th>Circuito</th><th>Computados</th><th>Votos Sen</th><th>Votos Dip</th><th>% diferencia</th></tr>'
for mesa in Mesa.objects.all():
    vot_dip = mesa.resultados.filter(opcion=UC, categoria='dip').first()
    vot_sen = mesa.resultados.filter(opcion=UC, categoria='sen').first()
    if vot_dip and vot_sen and vot_dip.votos and vot_sen.votos:
        vot_dip, vot_sen = vot_dip.votos, vot_sen.votos
        diferencia =  max(vot_dip, vot_sen)/min(vot_dip, vot_sen) 
        if diferencia > 1.3:
            diferencia = (diferencia - 1) * 100
            html += f"<tr><td><a href='{mesa.absolute_url}'>Mesa {mesa.numero}</a></td><td> {mesa.circuito} </td><td> {mesa.computados} </td><td> {vot_sen} </td><td> {vot_dip} </td><td> {diferencia:.1f}% </td></tr>"
html += '</table>'
    
display_html(HTML(html))


Mesa,Circuito,Computados,Votos Sen,Votos Dip,% diferencia
Mesa 00924,Circuito 0021A (003 - Almirante Brown),302,107,180,68.2%
Mesa 00308,Circuito 0668B (134 - Hurlingham),224,39,85,117.9%
Mesa 00099,Circuito 0258B (028 - Coronel Suárez),282,96,71,35.2%
Mesa 00100,Circuito 0258B (028 - Coronel Suárez),291,77,49,57.1%
Mesa 00102,Circuito 0258B (028 - Coronel Suárez),284,86,64,34.4%
Mesa 00103,Circuito 0258B (028 - Coronel Suárez),217,73,46,58.7%
Mesa 01006,Circuito 0022 (003 - Almirante Brown),272,116,88,31.8%
Mesa 00779,Circuito 0021 (003 - Almirante Brown),293,144,107,34.6%
Mesa 00226,Circuito 0668A (134 - Hurlingham),216,105,44,138.6%
Mesa 00335,Circuito 0667 (133 - Ituzaingo),246,61,5,1120.0%
